In [2]:
## Cargo las librerias
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
 

##Cargo las librerias de omdb
import omdb
import requests

##Cargo librerias de scraping web
import re 
from bs4 import BeautifulSoup
import requests

##Para archivos y directorios
import io
import os
import chardet

##Librerias de tiempo
import time
from datetime import datetime
import time

##Libreria del zip
from zipfile import ZipFile

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

##Librerías utilizadas para el muestreo de género de las peliculas
from collections import Counter
from random import uniform
from random import random
from bisect import bisect

###Librerias para el conteno de palabras (subtitulos)
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nl
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import string as st
from textblob import TextBlob

##Separar training y test
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn import preprocessing
##Metricas
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score

##Modelos
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score


##Graficos
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
##Importo el data set de películas
# Cargar la data
df_movies = pd.read_csv('./tmdb_5000_movies.csv', sep = ',')

print('df_movies: ',df_movies.shape[0])
print(df_movies.info())


df_movies:  4803
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count           

In [3]:
## Filtro las peliculas para que solo me traiga las que el idioma original sean en ingles
df_movies.original_language.value_counts()
df_movies = df_movies.loc[(df_movies.original_language == 'en') & (~df_movies.release_date.isnull()), :]
df_movies.shape

(4504, 20)

In [4]:
##Cambio las fechas a datetime
df_movies.release_date = pd.to_datetime(df_movies.release_date)
print('min fecha: ', min(df_movies.release_date))
print('max fecha: ',max(df_movies.release_date))
df_movies.head(5)

min fecha:  1916-09-04 00:00:00
max fecha:  2017-02-03 00:00:00


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
##creo el campo anio
df_movies['anio']=df_movies['release_date'].dt.year.astype(int)


In [ ]:
##Cargo el key de la api de omdb como default
omdb.set_default('apikey', 'c0b0cfca')


hora_ini = datetime.now()

data_base = pd.DataFrame()
##Consulto por película
for i in range(df_movies.shape[0]):
    try:
        peliculas = omdb.get(title=df_movies.iloc[i,6], media_type = 'movie', year = df_movies.iloc[i, 20], fullplot=True, tomatoes=True, timeout=60)
        df = pd.DataFrame.from_dict(json_normalize(peliculas), orient='columns')
        df['id_movie'] = df_movies.iloc[i, 3]
        data_base = pd.concat([data_base, df])
        time.sleep(1)
        if i%100 == 0:
            print(i)
            print('Duration: {}'.format(datetime.now() - hora_ini))
    except Exception as e:
        print("Unexpected error raised")
        print(e, "\n")
        continue

In [ ]:
df_movies.info()

In [ ]:
#Borro los que no pude detectar
data_movies = data_base.loc[~(data_base.id_movie.notnull()&data_base.title.isnull()),:]

In [ ]:
##Me fijo en las peliculas que no pudo detectar
faltantes = pd.merge(left=df_movies, right=data_movies, how= 'left',left_on='id', right_on='id_movie', indicator=True)
faltantes = faltantes.loc[faltantes['_merge'] == 'left_only', ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'anio']]


In [ ]:
faltantes.shape

In [ ]:
##Recupero los faltantes
##Cargo el key de la api de omdb como default
omdb.set_default('apikey', 'c0b0cfca')


hora_ini = datetime.now()

data_base = pd.DataFrame()
##Consulto por película
for i in range(faltantes.shape[0]):
    try:
        peliculas = omdb.get(title=faltantes.iloc[i,6], media_type = 'movie', year = faltantes.iloc[i, 20], fullplot=True, tomatoes=True, timeout=60)
        df = pd.DataFrame.from_dict(json_normalize(peliculas), orient='columns')
        df['id_movie'] = faltantes.iloc[i, 3]
        data_base = pd.concat([data_base, df])
        time.sleep(1)
        if i%10 == 0:
            print(i)
            print('Duration: {}'.format(datetime.now() - hora_ini))
    except Exception as e:
        print("Unexpected error raised")
        print(e, "\n")
        continue

In [ ]:
data_movies = pd.concat([data_movies, data_base.loc[data_base['title'].notnull(),:]])

In [ ]:
data_movies.info()

In [ ]:
##Guardo los imdb_id y los titulos
data_movies.loc[:,['title','imdb_id']].to_csv('imdb_id.csv', sep=';', index = False)

In [ ]:
##Guardo las peliculas
data_movies.to_csv('data_movies.csv', sep=';', index = False)

In [ ]:
data_movies.columns
data_movies.iloc[1,20]

In [2]:
##Importo el archivo
data_movies = pd.read_csv('./imdb_id2.txt', sep = ';')


In [3]:
data_movies.columns

Index(['title', 'imdb_id'], dtype='object')

In [10]:
###Traigo los subtitulos
hora_ini = datetime.now()
subtitulos_encontrados = pd.DataFrame()

head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
           'Chrome/59.0.3071.115 Safari/537.36'}

for i in range(2,100):
    try:
        s = requests.Session()
        
        ##Pagina para buscar el id de opensubtitles y para bajar el subtitulo
        search_url = 'https://www.opensubtitles.org/en/search/sublanguageid-eng/imdbid-'
        download_url = 'https://www.opensubtitles.org/en/download/vrf-108d030f/sub/'

        ##Hago el requests get cuando le paso el id de imdb
        #contenido = requests.get(search_url + data_movies.iloc[i,8], headers = head, auth=('alquimista1978','alqu1m1sta'))
        contenido = s.get(search_url + data_movies.iloc[i,1], headers = head, auth=('alquimista1978','alqu1m1sta'))


        ##Hago BeautifulSoup para estructurar la paguina
        soup = BeautifulSoup(contenido.text, 'html.parser')

        ##Busco la clase bnone y extraigo el href donde se encuentra el id de opensubtitles
        id_interno = soup.find('a', attrs={'class': 'bnone'}).get('href')
        ##Busco del href los numeros
        id_interno = re.findall('\\d+', id_interno)[0]
        
        time.sleep(np.random.uniform(1,10))
       
        ##genero el link para bajar el subtitulo con el id interno de opensubtitles
        #download_url = requests.get(download_url + id_interno, headers = head, auth=('alquimista1978','alqu1m1sta'))
        download_url = s.get(download_url + id_interno, headers = head, auth=('alquimista1978','alqu1m1sta'))
        
        ##Abro el zip del subtitulo 
        z = zipfile.ZipFile(io.BytesIO(download_url.content)) # Opening zip file

        ##Dentro del zip encuentro el archivo con extension srt
        srt = [m for m in z.namelist() if re.findall(pattern='.srt', string = m)][0]
        
        ##Guardo un resumen de lo encontrado
        s_econt = pd.DataFrame({'imdb_id': data_movies.iloc[i,1], 'title': data_movies.iloc[i,0], 'subtitulo':srt}, index= {i})
    
        subtitulos_encontrados = pd.concat([subtitulos_encontrados, s_econt])
        
        ##Del zip extraigo solo el subtitulo y lo reescribo con el id de imdb
        for f in z.filelist:
            if f.filename == srt:
                f.filename = data_movies.iloc[i,8] +'.srt'
                z.extract(member = f, path='./Subtitulos/')
        
        time.sleep(np.random.uniform(10,20))
        
        ##Control de tiempo, cuantos va ejecutando y cuantos subtitulos encontro
        if i%10 == 0:
            print('Cantidad de registros procesados:', i)
            print('Tiempo de ejecucion: {}'.format(datetime.now() - hora_ini))
            print('Cantidad de subtitulos encontrados:', len(np.unique(subtitulos_encontrados.subtitulo)))
    except Exception as e:
        print("Unexpected error raised")
        print(e, "\n")
        print(download_url)
        time.sleep(np.random.uniform(10,20))
        continue    

Unexpected error raised
name 'zipfile' is not defined 

<Response [200]>


KeyboardInterrupt: 

In [ ]:
#Vemos el contenido renderizado
display(HTML(download_url.text))

In [ ]:
###Traigo los subtitulos de otra forma
hora_ini = datetime.now()
subtitulos_encontrados = pd.DataFrame()

head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
           'Chrome/59.0.3071.115 Safari/537.36'}

for i in range(171, data_movies.shape[0]):
    try:
        s = requests.Session()
        
        ##Pagina para buscar el id de opensubtitles y para bajar el subtitulo
        search_url = 'https://www.opensubtitles.org/en/search/sublanguageid-eng/imdbid-'
        download_url = 'https://www.opensubtitles.org/en/download/vrf-108d030f/sub/'

        ##Hago el requests get cuando le paso el id de imdb
        #contenido = requests.get(search_url + data_movies.iloc[i,8], headers = head, auth=('alquimista1978','alqu1m1sta'))
        contenido = s.get(search_url + data_movies.iloc[i,8], headers = head, auth=('alquimista1978','alqu1m1sta'))


        ##Hago BeautifulSoup para estructurar la paguina
        soup = BeautifulSoup(contenido.text, 'html.parser')

        ##Busco la clase bnone y extraigo el href donde se encuentra el id de opensubtitles
        id_interno = soup.find('a', attrs={'class': 'bnone'}).get('href')
        ##Busco del href los numeros
        id_interno = re.findall('\\d+', id_interno)[0]
        
        time.sleep(np.random.uniform(1,2))
       
        ##Guardo un resumen de lo encontrado
        s_econt = pd.DataFrame({'imdb_id': data_movies.iloc[i,8], 'title': data_movies.iloc[i,20], 'download_url':download_url + id_interno}, index= {i})
    
        subtitulos_encontrados = pd.concat([subtitulos_encontrados, s_econt])
        
        ##Control de tiempo, cuantos va ejecutando y cuantos subtitulos encontro
        if i%10 == 0:
            print('Cantidad de registros procesados:', i)
            print('Tiempo de ejecucion: {}'.format(datetime.now() - hora_ini))
            
    except Exception as e:
        print("Unexpected error raised")
        print(e, "\n")
        print(download_url)
        continue    

In [ ]:
subtitulos_encontrados.to_csv('download.csv', sep=';', index = False)